In [1]:
import pandas as pd
import json
import time
import matplotlib
import matplotlib.pyplot as plt
import re
import numpy as np
from nltk.tokenize import RegexpTokenizer
import nltk
# Import all the libraries required
import os
import pandas as pd
import matplotlib.cm as cm
from matplotlib.colors import rgb2hex
from descartes import PolygonPatch
from shapely.geometry import Polygon, MultiPolygon
# Importing Gensim
import gensim
from gensim import corpora
import time
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## What topics come up from the tweets that talk about Hillary and Trump separately?

In [2]:
tweets_data = []

#with open('tweets_reduced_10000.jsons', 'r') as tweets_file:
#with open('geotagged_tweets_20160812-0912.jsons', 'r') as tweets_file:
#with open('customTweets.jsons', 'r') as tweets_file:
#with open('customTweetsWithSentiment2.jsons', 'r') as tweets_file:
with open('customTweetsWithSentiment3.jsons', 'r') as tweets_file:
    for line in tweets_file:
        try:
            tweet = json.loads(line)
            tweets_data.append(tweet)
        except Exception as e:
            print (e)
            continue

### Splitting tweets

In [3]:
tweets_trump = [tweet['text'] for tweet in tweets_data if tweet['talk'] == 'trump' and tweet['place'] != None and tweet['place']['country_code'] == 'US']
tweets_hillary = [tweet['text'] for tweet in tweets_data if tweet['talk'] == 'hillary' and tweet['place'] != None and tweet['place']['country_code'] == 'US']

In [4]:
tweets_pro_trump = [tweet['text'] for tweet in tweets_data if (tweet['talk'] == 'trump' and tweet['sentiment'] == 'Positive') or (tweet['talk'] == 'hillary' and tweet['sentiment'] == 'Negative') and tweet['place'] != None and tweet['place']['country_code'] == 'US']
tweets_pro_hillary = [tweet['text'] for tweet in tweets_data if (tweet['talk'] == 'hillary' and tweet['sentiment'] == 'Positive') or (tweet['talk'] == 'trump' and tweet['sentiment'] == 'Negative') and tweet['place'] != None and tweet['place']['country_code'] == 'US']

In [5]:
print('Trump only:   ' + str(len(tweets_trump)))
print('Hillary only: ' + str(len(tweets_hillary)))

Trump only:   314050
Hillary only: 128836


In [6]:
print('Pro Trump only:   ' + str(len(tweets_pro_trump)))
print('Pro Hillary only: ' + str(len(tweets_pro_hillary)))

Pro Trump only:   277971
Pro Hillary only: 198395


### Cleaning

In [7]:
# A function that removes the hyperlinks from the tweet's content.
def remove_link(tweet):
    return re.sub(r"http\S+", "", tweet)

def remove_hashtags(tweet):
    return re.sub('#', '', tweet)

def remove_entire_hashtags(tweet):
    return re.sub(r'(?:\s|^)#[A-Za-z0-9\-\.\_]+(?:\s|$)', ' ', tweet).strip()

def remove_mentions(tweet):
    return " ".join(re.sub("(?:\@|https?\://)\S+"," ",tweet).split())


def remove_non_az_characters(tweet):
    return " ".join(re.sub('[^a-zA-Z]'," ",tweet).split())

In [8]:
remove_mentions("hello @you how is @Albert? #angry https://google.com ")

'hello how is #angry'

In [9]:
remove_non_az_characters("hello 45 @you how is @Albert? #angry https://google.com ")

'hello you how is Albert angry https google com'

In [10]:
remove_entire_hashtags('#start hello #angry you #bye')

'hello you'

In [11]:
# https://pypi.org/project/emoji/

import emoji

def extract_emojis(str):
    return ''.join(c for c in str if c in emoji.UNICODE_EMOJI)

def text_without_emojis(str):
    return ''.join(c for c in str if c not in emoji.UNICODE_EMOJI)

text_with_emoji = "Come to Jesus meeting!!!! What on earth is that supposed to be? 😔 https://t.co/a3lOpTtFig"
print(text_with_emoji)
print(extract_emojis(text_with_emoji))
print(text_without_emojis(text_with_emoji))

Come to Jesus meeting!!!! What on earth is that supposed to be? 😔 https://t.co/a3lOpTtFig
😔
Come to Jesus meeting!!!! What on earth is that supposed to be?  https://t.co/a3lOpTtFig


In [12]:
# Create a set of stopwords
stop = set(stopwords.words('english'))

# Create a set of punctuation words 
exclude = set(string.punctuation) 

# This is the function makeing the lemmatization
lemma = WordNetLemmatizer()

# In this function we perform the entire cleaning
def clean(doc):
    doc = doc.lower()
    doc = remove_link(doc)
    #print('without links:', doc)
    doc = remove_mentions(doc)
    #print('without mentions:', doc)
    doc = text_without_emojis(doc)
    #print('without emojis:', doc)
    doc = remove_entire_hashtags(doc)
    #print('without hashtags:', doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #print('stop free:', doc)
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    #print('punc_free:', doc)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    #print('normalized:', normalized)
    normalized = remove_non_az_characters(normalized)
    #print('non-az removed:', normalized)
    normalized = " ".join(word for word in normalized.split() if len(word) > 3)
    return normalized

In [13]:
clean('I like reading, so I read 45. @remy: This is @hello waaaaayyyy too much. For you!!!!!! #angry https://google.com 😔')

'like reading read waaaaayyyy much'

In [14]:
clean('@remy: This is @hello waaaaayyyy too much. For you!!!!!! #angry https://google.com 😔')

'waaaaayyyy much'

### Clean words like 'trump' or 'hillary'

In [15]:
def removeKeyWords(candidates, tweets_text):
    keyWords = {
        'hillary' : ['hillary', 'clinton', 'imwithher'],
        'trump' : ['donald', 'trump']
    }
    for candidate in candidates:
        words = keyWords[candidate]
        for idx, txt in enumerate(tweets_text):
            tweet = ''
            for word in txt.lower().split():
                found = False
                for keyword in words:
                    if keyword in word:
                        found = True
                        break
                if not found:    
                    tweet += word + ' '
            tweets_text[idx] = tweet.strip()
    return tweets_text  

In [16]:
tweets_text = ['hello hillarybla Hillary clinton im']
removeKeyWords(['hillary'], tweets_text)

['hello im']

In [17]:
clean('hello hillarybla Hillary clinton im')

'hello hillarybla hillary clinton'

In [18]:
arr = removeKeyWords(['hillary'], [clean('hello hillarybla aaaaa Hillary clinton im')])


In [19]:
def topicModelingCleanedTweets(tweets_text, candidates):
    # This is the clean corpus.
    doc_clean = [removeKeyWords(candidates, [clean(doc)])[0].split() for doc in tweets_text] 
    # Creating the term dictionary of our courpus, where every unique term is assigned an index
    dictionary = corpora.Dictionary(doc_clean)

    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

    NUM_TOPICS = 4

    start_time = time.time()


    # Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel

    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=NUM_TOPICS, id2word = dictionary, passes=100)

    elapsed_time = time.time() - start_time
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

    # Print 4 topics and describe then with 4 words.
    topics = ldamodel.print_topics(num_topics=NUM_TOPICS, num_words=4)

    i=0
    for topic in topics:
        print ("Topic",i ,"->", topic)     
        i+=1

In [20]:
topicModelingCleanedTweets(tweets_trump, ['trump'])

02:12:44
Topic 0 -> (0, '0.038*"today" + 0.029*"pressure" + 0.029*"wind" + 0.029*"rain"')
Topic 1 -> (1, '0.020*"know" + 0.015*"like" + 0.011*"putin" + 0.010*"said"')
Topic 2 -> (2, '0.019*"america" + 0.014*"great" + 0.012*"deplorable" + 0.012*"make"')
Topic 3 -> (3, '0.024*"vote" + 0.016*"american" + 0.012*"love" + 0.010*"white"')


In [21]:
topicModelingCleanedTweets(tweets_hillary, ['hillary'])

00:46:52
Topic 0 -> (0, '0.020*"deplorable" + 0.020*"health" + 0.014*"stop" + 0.012*"racist"')
Topic 1 -> (1, '0.016*"like" + 0.015*"pneumonia" + 0.011*"know" + 0.010*"sick"')
Topic 2 -> (2, '0.015*"well" + 0.011*"need" + 0.010*"america" + 0.010*"woman"')
Topic 3 -> (3, '0.014*"cant" + 0.011*"like" + 0.010*"never" + 0.009*"said"')


## Topic analysis pro-trump vs topic analysis pro-hillary

In [22]:
topicModelingCleanedTweets(tweets_pro_trump, ['trump','hillary'])

02:28:15
Topic 0 -> (0, '0.039*"today" + 0.028*"pressure" + 0.028*"wind" + 0.028*"rain"')
Topic 1 -> (1, '0.013*"like" + 0.011*"know" + 0.011*"supporter" + 0.010*"people"')
Topic 2 -> (2, '0.015*"america" + 0.013*"american" + 0.012*"vote" + 0.012*"president"')
Topic 3 -> (3, '0.017*"deplorable" + 0.017*"putin" + 0.011*"health" + 0.009*"leader"')


In [23]:
topicModelingCleanedTweets(tweets_pro_hillary, ['trump','hillary'])

01:14:00
Topic 0 -> (0, '0.018*"like" + 0.014*"vote" + 0.011*"deplorable" + 0.011*"woman"')
Topic 1 -> (1, '0.024*"pneumonia" + 0.024*"think" + 0.013*"racist" + 0.009*"tweet"')
Topic 2 -> (2, '0.011*"time" + 0.011*"love" + 0.010*"need" + 0.008*"come"')
Topic 3 -> (3, '0.011*"president" + 0.011*"never" + 0.009*"know" + 0.009*"email"')


In [24]:
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()
lemma.lemmatize('nevertrump')

'nevertrump'